<a href="https://colab.research.google.com/github/BhauArage/Fake-News/blob/main/Fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Necessary Libraries and Packages**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Core packages for text processing.

import string
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Loading some sklearn packages for modelling.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, accuracy_score

# Some packages for word clouds and NER.

from wordcloud import WordCloud, STOPWORDS
from collections import Counter, defaultdict

# Loading pytorch packages.

import torch
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Using GPU**

In [ ]:
if torch.cuda.is_available():    
    device = torch.device('cuda')    
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
print('Using device:', device)

Using device: cuda


**Reading Dataset**

In [ ]:
test=pd.read_csv('/test.csv')
train=pd.read_csv('/train.csv')


print(f'Number of training tweets: {train.shape[0]}\n')
print(f'Number of training tweets: {test.shape[0]}\n')

display(train.sample(10))

Number of training tweets: 20800

Number of training tweets: 5200



,id,title,author,text,label
3786,3786,In Consideration of the Supreme Importance of ...,Les Visible,"By Les Visible on November 4, 2016 Visible Ori...",1
8814,8814,Long-term Effects of the Presidential Election,jhamilton,"November 10, 2016 Long-term Effects of the Pre...",1
5856,5856,There’s More to Rosé Than You May Think - The ...,Eric Asimov,"Summer arrives, and along with it, rosé and en...",0
9881,9881,"Hillary Clinton and Donald Trump, Ages 68 and ...",Patrick Healy,Donald J. Trump and Hillary Clinton have been ...,0
14893,14893,The Other 13 Women Testifying Against Cosby .....,Graham Bowley and Sydney Ember,Some knew him for only a day. Others built fri...,0
166,166,Takata Chief Executive to Resign as Financial ...,Jonathan Soble,TOKYO — The chief executive of Takata said ...,0
4694,4694,Millions of South Koreans Rise Up Against Shad...,Madeline,Your News Wire \nSouth Koreans are rising up i...,1
19600,19600,Fox News Channel Cancels ’Red Eye’ - Breitbart,Daniel Nussbaum,Fox News Channel has canceled its news pro...,0
1674,1674,30 Civilians Die In US Airstrike Called ‘To Pr...,The Guardian,Videos 30 Civilians Die In US Airstrike Called...,1
14968,14968,Pee Wee Football Team Banned From League After...,Kim Smith,Lifesize Noah’s Ark Is Massive Success Despite...,1


In [ ]:
labels = train['label'].values
idx = len(labels)
combined = pd.concat([train, test])
combined = combined.fillna('no data')
df = combined['title'] + ' ' + combined['author']
combined = df.values

In [ ]:
combined

array(['House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell Lucus',
       'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart Daniel J. Flynn',
       'Why the Truth Might Get You Fired Consortiumnews.com', ...,
       'California Today: What, Exactly, Is in Your Sushi? - The New York Times Mike McPhate',
       '300 US Marines To Be Deployed To Russian Border In Norway no data',
       'Awkward Sex, Onscreen and Off - The New York Times Teddy Wayne'],
      dtype=object)

In [ ]:
Tokenizer=BertTokeizer.from_pretrained('bert-base-case')

NameError: ignored